<a href="https://colab.research.google.com/github/katrinag2004/dsproject2/blob/main/cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ast         
import pandas as pd

SRC  = "/Users/codyliddle/Desktop/UVA Year 3/DS Final Project/dsproject2/tracks_features.csv"
DEST = "processed_local_data.csv"      

# 1. Load only the two columns we care about
df = pd.read_csv(SRC, usecols=["artists", "artist_ids"])

# 2. Parse the stringified‑list cells → real lists
def to_list(cell):
    """
    ast.literal_eval safely turns "['A','B']" → ['A','B'].
    If the cell is already a list (rare), it just returns it.
    """
    if isinstance(cell, list):
        return cell
    try:
        return ast.literal_eval(cell)
    except Exception:
        # fallback: treat as single‑item list
        return [str(cell)]

df["artists"]     = df["artists"].apply(to_list)
df["artist_ids"]  = df["artist_ids"].apply(to_list)

# 3. Sanity‑check lengths (keep rows where counts match)
mask = df.apply(lambda r: len(r["artists"]) == len(r["artist_ids"]), axis=1)
if not mask.all():
    print("Dropping rows where artist list and id list lengths differ.")
df = df[mask]

# 4. Explode so we get *one* artist per row
df = df.explode(["artists", "artist_ids"], ignore_index=True)

# 5. Rename & clean
df = (
    df
    .rename(columns={"artists": "artist_name", "artist_ids": "artist_id"})
    .assign(
        artist_name=lambda x: x["artist_name"].str.strip(),
        artist_id=lambda x: x["artist_id"].str.strip()
    )
    .drop_duplicates(subset="artist_id", keep="first")  # keep first occurrence
    .sort_values("artist_name")
)

# 6. Save
df.to_csv(DEST, index=False)
print(f"Saved {len(df):,} unique artists to {DEST}")


✅  Saved 140,449 unique artists to processed_local_data.csv


In [20]:
from google.colab import files
files.download("modified_songs3.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>